# Rust-Python Binding Notebook Course

In [ ]:
import blogpost_ffi
import os
import pyarrow as pa
import time
import tracing
from opentelemetry.trace.propagation.tracecontext import (
    TraceContextTextMapPropagator,
)

In [ ]:
pa.array([])
value = [1] * 100_000_000

In [ ]:
# Default implementation

print(f"---Default Implementation---")

start_time = time.time()

array = blogpost_ffi.create_list(value)

print(f"default: {time.time() - start_time:.3f}s")

In [ ]:
# PyBytes Implementation

print(f"---PyBytes Implementation---")

value_bytes = bytes(value)

start_time = time.time()

array = blogpost_ffi.create_list_bytes(value_bytes)

print(f"bytes: {time.time() - start_time:.3f}s")


In [ ]:
# Arrow Implementation

print(f"---Arrow Implementation---")

value_arrow = pa.array(value, type=pa.uint8())

start_time = time.time()

array = blogpost_ffi.create_list_arrow(value_arrow)

print(f"arrow: {time.time() - start_time:.3f}s")

In [1]:
# Debugging eyre

print(f"---Eyre error---")

# Try:
# array = blogpost_ffi.create_list_arrow(1)
#
## This error panics the whole program and is therefore uncatchable.
ERROR_WITHOUT_EYRE = """
thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }', src/lib.rs:45:62
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Traceback (most recent call last):
  File "/home/peter/Documents/work/blogpost_ffi/test_script.py", line 79, in <module>
    array = blogpost_ffi.create_list_arrow(1)
pyo3_runtime.PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }
"""


# Eyre result
array = blogpost_ffi.create_list_arrow_eyre(1)

---Eyre error---


NameError: name 'blogpost_ffi' is not defined

In [ ]:
# Eyre default

def abc():
    assert False, "I have no idea what is wrong"

In [ ]:
print(f"---Eyre no traceback---")

array = blogpost_ffi.call_func_eyre(abc)

In [ ]:
# Eyre traceback
print(f"---Eyre traceback---")

array = blogpost_ffi.call_func_eyre_traceback(abc)

In [ ]:
# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.unbounded_memory_growth()

In [ ]:

# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.bounded_memory_growth()

In [ ]:
# GIL Lock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_lock()

In [ ]:
# GIL unlock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_unlock()

In [ ]:
# Global tracing
print(f"---Global tracing---")

def abc(cx):
    propagator = TraceContextTextMapPropagator()
    context = propagator.extract(carrier=cx)

    with tracing.tracer.start_as_current_span(
        name="Python_span", context=context
    ) as child_span:
        child_span.add_event("in Python!")
        output = {}
        tracing.propagator.inject(output)
        time.sleep(2)
    return output


array = blogpost_ffi.global_tracing(abc)